In [125]:
import numpy as np
import helpers as h
import networkx as nx
import pickle

import zipfile
from io import BytesIO
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import MultiPoint
from shapely.geometry import Point
import pyproj

import momepy


In [126]:
#accessed zipped geojson - please note this is a subset of the original data. The data is a spatial subset for two neighbourhoods - Osdorp Midden & Nieuwmarkt
gdf_path = "data/accessibility_dataset.gpkg"
gdf = gpd.read_file(gdf_path)

In [127]:
#settings

max_curb_height = 0.04  # m
min_sidewalk_width = 0.80  # m

# Boundaries between the final colors (in meters)
width_1 = 0.6
width_2 = 0.8
width_3 = 1.0
width_4 = 1.2
width_5 = 1.4
width_6 = 1.6

### Set hard limits (remove inaccessible curbs based on height)

In [128]:
gdf['include'] = 1 

In [129]:
# Check if there are any curb height crossings without max height

gdf[gdf['crossing_type'] == 'curb_height']['curb_height_max'].value_counts(dropna=False)

curb_height_max
0.08    12382
0.06     8757
0.04     5925
Name: count, dtype: int64

In [130]:
# Don't include crossings with curbs that are too high
gdf.loc[gdf['curb_height_max'] > max_curb_height, 'include'] = 0

In [131]:
# Check if the right amount of paths are included
print(gdf['curb_height_max'].value_counts(dropna=False))
print(gdf['include'].value_counts(dropna=False))

curb_height_max
NaN     54084
0.08    12382
0.06     8757
0.04     5925
Name: count, dtype: int64
include
1    60009
0    21139
Name: count, dtype: int64


### Set hard limits (remove inaccessible curbs based on width)

In [132]:
# Give crossings a width
gdf.loc[gdf['crossing'] == 'Yes', 'obstacle_free_width_float'] = width_6
gdf.loc[gdf['crossing'] == 'Yes', 'width_fill'] = 4

# Give bike paths a width
gdf.loc[~gdf['bikepath_id'].isnull(), 'obstacle_free_width_float'] = width_5
gdf.loc[~gdf['bikepath_id'].isnull(), 'width_fill'] = 4

# Give walk bike connections a width
gdf.loc[gdf['walk_bike_connection'] == 'Yes', 'obstacle_free_width_float'] = width_5
gdf.loc[gdf['walk_bike_connection'] == 'Yes', 'width_fill'] = 4

# Give walk public transport stop connections a width if unknown
gdf.loc[(gdf['walk_pt_connection'] == 'Yes') & gdf['obstacle_free_width_float'].isnull(), 'width_fill'] = 4
gdf.loc[(gdf['walk_pt_connection'] == 'Yes') & gdf['obstacle_free_width_float'].isnull(), 'obstacle_free_width_float'] = width_2

In [133]:
# Check if there are any remaining paths without width
gdf.loc[(gdf['obstacle_free_width_float'].isnull()) & (gdf['public_transport_stop'] == 'No')]

,path_type,length,sidewalk_id,bikepath_id,obstacle_free_width,obstacle_free_width_float,width_fill,crossing,crossing_type,curb_height_max,walk_bike_connection,walk_pt_connection,public_transport_stop,stop_type,stop_name,stop_placement_type,wheelchair_accessible,geometry,include


In [134]:
# Don't include paths that are too narrow
gdf.loc[gdf['obstacle_free_width_float'] < min_sidewalk_width, 'include'] = 0

In [135]:
# Check if the right amount of paths are included
print(gdf['obstacle_free_width_float'].value_counts())
print(gdf['include'].value_counts(dropna=False))

obstacle_free_width_float
1.60    65159
1.40     6920
0.40     2292
1.00     1823
0.80     1805
1.20     1725
0.60     1358
0.90        6
1.10        6
1.15        4
1.30        4
1.50        4
1.45        4
2.40        4
1.80        3
0.50        2
0.91        2
3.15        2
3.20        2
2.00        2
1.65        2
1.05        2
1.25        2
Name: count, dtype: int64
include
1    56357
0    24791
Name: count, dtype: int64


### Further preprocessing

In [136]:
## Further preprocessing , to get columns for the different objectives

gdf_accessible = gdf.copy()
gdf_accessible['id'] = 'id'+'_'+(gdf.reset_index()['index']).astype(str)

pivoted = gdf_accessible.pivot_table(
    index='id',     
    columns='path_type',      
    values='length',          
    aggfunc='sum')


pivoted.reset_index(inplace=True)
gdf_accessible_pivot = gdf_accessible.merge(pivoted, on='id', how='left')
gdf_accessible_pivot['crossing'] = gdf_accessible_pivot['crossing'].map({"Yes":1, "No":0})
gdf_accessible_pivot['public_transport_stop'] = gdf_accessible_pivot['public_transport_stop'].map({"Yes":1, "No":0})
gdf_accessible_pivot['walk_pt_connection'] = gdf_accessible_pivot['walk_pt_connection'].map({"Yes":1, "No":0})

gdf_accessible_pivot = gdf_accessible_pivot[['path_type','length','walk','walk_bike_connection_x','crossing','bike','obstacle_free_width_float','curb_height_max','walk_pt_connection','public_transport_stop','stop_name','geometry']]
gdf_accessible_pivot = gdf_accessible_pivot.rename(columns={'walk_bike_connection_x':'walk_bike_connection'})
gdf_accessible_pivot = gdf_accessible_pivot.rename(columns={'public_transport_stop_y':'public_transport_stop'})

gdf_accessible_pivot['oneway'] = np.where(gdf_accessible_pivot['path_type']=='walk',False,True)
gdf_accessible_pivot = GeoDataFrame(gdf_accessible_pivot, crs="EPSG:28992", geometry='geometry')

#### Add distance to public transport per edge

In [137]:
centroids = (
    gdf_accessible_pivot[gdf_accessible_pivot['public_transport_stop']==1].groupby("stop_name")
    .apply(h.calculate_centroid)
    .reset_index(name="centroid_geometry")
)

# Convert to GeoDataFrame
centroids_gdf = gpd.GeoDataFrame(centroids, geometry="centroid_geometry",crs="EPSG:28992")

# Ensure both GeoDataFrames share the same CRS
if gdf_accessible_pivot.crs != centroids_gdf.crs:
    centroids_gdf = centroids_gdf.to_crs(gdf_accessible_pivot.crs)

gdf_accessible_pivot["distance_to_pt_stops_float"] = gdf.apply(
    lambda row: h.shortest_distance(row, centroids_gdf["centroid_geometry"]),
    axis=1,
)

C:\Users\krishn001\AppData\Local\Temp\ipykernel_30016\913432006.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(h.calculate_centroid)
c:\Users\krishn001\AppData\Local\anaconda3\envs\rl\Lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


In [138]:
## convert column distance_to_pt_stops to <300m and >= 300m

gdf_accessible_pivot['distance_to_pt_stops'] = 0
gdf_accessible_pivot.loc[(gdf_accessible_pivot['distance_to_pt_stops_float']>=0) & (gdf_accessible_pivot['distance_to_pt_stops_float']<400),'distance_to_pt_stops' ] = '<400'
gdf_accessible_pivot.loc[(gdf_accessible_pivot['distance_to_pt_stops_float']>=400) ,'distance_to_pt_stops' ] = '>=400'

C:\Users\krishn001\AppData\Local\Temp\ipykernel_30016\293227496.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<400' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gdf_accessible_pivot.loc[(gdf_accessible_pivot['distance_to_pt_stops_float']>=0) & (gdf_accessible_pivot['distance_to_pt_stops_float']<400),'distance_to_pt_stops' ] = '<400'


In [139]:
gdf_accessible_pivot = gdf_accessible_pivot.fillna(0)

### Spatial subsets per neighbourhood

In [140]:
# Nieuwmarkt 

min_lon_nwmkt, max_lon_nwmkt = 121000, 124000
min_lat_nwmkt, max_lat_nwmkt = 486000, 488000

df_network_nwmkt = gdf_accessible_pivot.cx[min_lon_nwmkt:max_lon_nwmkt, min_lat_nwmkt:max_lat_nwmkt]

# Osdorp Midden 

min_lon_osdpm, max_lon_osdpm = 112000, 116000
min_lat_osdpm, max_lat_osdpm = 484000, 486500

df_network_osdpm = gdf_accessible_pivot.cx[min_lon_osdpm:max_lon_osdpm, min_lat_osdpm:max_lat_osdpm]

#converts the dataframe to a multigraph network

G_nwmkt = momepy.gdf_to_nx(df_network_nwmkt, approach="primal", multigraph=True, directed=True, oneway_column="oneway")
G_osdpm = momepy.gdf_to_nx(df_network_osdpm, approach="primal", multigraph=True, directed=True, oneway_column="oneway")

gdf_nwmkt_connected, G_nwmkt_connected = h.return_connected_networks(G_nwmkt)
gdf_osdpm_connected, G_osdpm_connected = h.return_connected_networks(G_osdpm)


G_nwmkt_pt = momepy.gdf_to_nx(df_network_nwmkt[df_network_nwmkt['distance_to_pt_stops']=='<400'], approach="primal", multigraph=True, directed=True, oneway_column="oneway")
G_osdpm_pt = momepy.gdf_to_nx(df_network_osdpm[df_network_osdpm['distance_to_pt_stops']=='<400'], approach="primal", multigraph=True, directed=True, oneway_column="oneway")

gdf_nwmkt_connected_pt, G_nwmkt_connected_pt = h.return_connected_networks(G_nwmkt_pt)
gdf_osdpm_connected_pt, G_osdpm_connected_pt = h.return_connected_networks(G_osdpm_pt)

C:\Users\krishn001\AppData\Local\Temp\ipykernel_30016\635274422.py:17: RuntimeWarning: The given network consists of multiple geometry types. This can lead to unexpected behaviour. The intended usage of the conversion function is with networks made of LineStrings only.
  G_nwmkt = momepy.gdf_to_nx(df_network_nwmkt, approach="primal", multigraph=True, directed=True, oneway_column="oneway")
C:\Users\krishn001\AppData\Local\Temp\ipykernel_30016\635274422.py:18: RuntimeWarning: The given network consists of multiple geometry types. This can lead to unexpected behaviour. The intended usage of the conversion function is with networks made of LineStrings only.
  G_osdpm = momepy.gdf_to_nx(df_network_osdpm, approach="primal", multigraph=True, directed=True, oneway_column="oneway")
c:\Users\krishn001\peer_ams\helpers.py:63: RuntimeWarning: The given network consists of multiple geometry types. This can lead to unexpected behaviour. The intended usage of the conversion function is with networks 

### Save relevant graphs

In [141]:
adj_matrix_nwmkt = h.create_adj_matrix(gdf_nwmkt_connected)
adj_matrix_osdpm = h.create_adj_matrix(gdf_osdpm_connected)

## adj_matrix for graphs with edges where PT stop is less than 300 m

adj_matrix_nwmkt_pt = h.create_adj_matrix(gdf_nwmkt_connected_pt)
adj_matrix_osdpm_pt = h.create_adj_matrix(gdf_osdpm_connected_pt)

# dictionary for mapping node ids

node_dict_nwmkt =  h.create_node_dict(gdf_nwmkt_connected)
nodde_dict_osdpm = h.create_node_dict(gdf_osdpm_connected)

### Link removal for MVP 1 (not used)

In [142]:
gdf_link_removal = gdf_osdpm_connected_pt.copy()

In [143]:
#remove edge with point - 52.362872, 4.788045

lat = 52.354668  
lon = 4.798792 

gdf_crs = gdf_link_removal.crs  # Get the CRS of the GeoDataFrame

# Define a transformer to convert lat, lon (WGS84) to the CRS of your GeoDataFrame
transformer = pyproj.Transformer.from_crs("EPSG:4326", gdf_crs, always_xy=True)


x, y = transformer.transform(lon, lat)  # Reverse order for lat, lon -> lon, lat
point = Point(x, y)
gdf_link_removal['distance'] = gdf_link_removal.geometry.distance(point)
gdf_filtered = gdf_link_removal[gdf_link_removal['distance'] > 0.2]
gdf_filtered = gdf_filtered.drop(columns='distance')

c:\Users\krishn001\AppData\Local\anaconda3\envs\rl\Lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


In [144]:
#Osdorp Midden
gdf_filtered.to_file("preprocessed_data/gdf_filtered.gpkg", driver='GPKG')
adj_matrix_filtered = h.create_adj_matrix(gdf_nwmkt_connected)

# Save the adjacency matrix to a text file
filename = 'preprocessed_data/adj_matrix_filtered.txt'
with open(filename, 'w') as f:
    for origin, destinations in adj_matrix_filtered.items():
        for destination, properties in destinations.items():
            f.write(f"{origin} {destination} {properties}\n")

### Store all files

In [145]:
# Nieuwmkart

gdf_nwmkt_connected.to_file("preprocessed_data/gdf_nwmkt_connected.gpkg", driver='GPKG')
gdf_nwmkt_connected_pt.to_file("preprocessed_data/gdf_nwmkt_connected_pt.gpkg", driver='GPKG')

with open('preprocessed_data/graph_nwmkt.gpickle', 'wb') as f:
    pickle.dump(G_nwmkt_connected, f, pickle.HIGHEST_PROTOCOL)

with open('preprocessed_data/graph_nwmkt_pt.gpickle', 'wb') as f:
    pickle.dump(G_nwmkt_connected_pt, f, pickle.HIGHEST_PROTOCOL)

# Save the adjacency matrix to a text file
filename = 'preprocessed_data/adjacency_matrix_nwmkt.txt'
with open(filename, 'w') as f:
    for origin, destinations in adj_matrix_nwmkt.items():
        for destination, properties in destinations.items():
            f.write(f"{origin} {destination} {properties}\n")

filename = 'preprocessed_data/adjacency_matrix_nwmkt_pt.txt'
with open(filename, 'w') as f:
    for origin, destinations in adj_matrix_nwmkt_pt.items():
        for destination, properties in destinations.items():
            f.write(f"{origin} {destination} {properties}\n")

with open('preprocessed_data/node_dict_nwmkt.pickle', 'wb') as f:
    pickle.dump(node_dict_nwmkt, f)

In [146]:
# Osdorp Midden

gdf_osdpm_connected.to_file("preprocessed_data/gdf_osdpm_connected.gpkg", driver='GPKG')
gdf_osdpm_connected_pt.to_file("preprocessed_data/gdf_osdpm_connected_pt.gpkg", driver='GPKG')

with open('preprocessed_data/graph_osdpm.gpickle', 'wb') as f:
    pickle.dump(G_osdpm_connected, f, pickle.HIGHEST_PROTOCOL)

with open('preprocessed_data/graph_osdpm_pt.gpickle', 'wb') as f:
    pickle.dump(G_osdpm_connected_pt, f, pickle.HIGHEST_PROTOCOL)

# Save the adjacency matrix to a text file
filename = 'preprocessed_data/adjacency_matrix_osdpm.txt'
with open(filename, 'w') as f:
    for origin, destinations in adj_matrix_osdpm.items():
        for destination, properties in destinations.items():
            f.write(f"{origin} {destination} {properties}\n")

filename = 'preprocessed_data/adjacency_matrix_osdpm_pt.txt'
with open(filename, 'w') as f:
    for origin, destinations in adj_matrix_osdpm_pt.items():
        for destination, properties in destinations.items():
            f.write(f"{origin} {destination} {properties}\n")

with open('preprocessed_data/node_dict_osdpm.pickle', 'wb') as f:
    pickle.dump(nodde_dict_osdpm, f)